Wczytajmy oczyszczone dane

In [3]:
import json
from datetime import datetime

In [4]:
folder = 'data'

with open(folder + '/events.json') as file:
    events_list = json.load(file)

with open(folder + '/products.json') as file:
    products_list = json.load(file)

with open(folder + '/users.json') as file:
    users_list = json.load(file)


Jedną jednostką uczącą rekurencyjnej sieci neuronowej będzie lista wydarzeń w sesji.

Na początku wyodrębniamy poszczególne sesje.

Następnie przygotujmy atrybuty do modelu.
- events_list:
    - timestamp -> czas od poprzedniego wydarzenia
    - event_type -> bool isBuying: =="BUY_PRODUCT"
    - offered_discount
    - User
    - Product
- users_list:
    - city -> hot 1
- products_list:
    - category -> hot 1
    - price

In [17]:
#definition of classes

class User:
    def __init__(self, city):
        self.city = city

    def get_city(self):
        return self.city


users = dict()

for user in users_list:
    users[user['user_id']] = User(user['city'])


class Product:
    def __init__(self, product_id, price, category):
        self.product_id = product_id
        self.price = price
        self.category = category


# def to_categorical(self, category_list):
#         # one hot encode
#         encoded = to_categorical(category_list)
#         print(encoded)
#         # invert encoding
#         inverted = argmax(encoded[0])
#         print(inverted)

products = dict()

for product in products_list:
    products[product['product_id']] = Product(product['product_id'], product['price'], product['category_path'])


class Event:
    def __init__(self, time, user_city, product_price, product_category, event_type, discount):
        self.time = time.total_seconds()
        self.user_city = user_city
        self.product_price = product_price
        self.product_category = product_category
        self.bought = event_type == "BUY_PRODUCT"
        self.discount = discount

# 
# class Session:
#     def __init__(self, events_list):
#         self.events_list = events_list



In [26]:
sessions_list = []

last_session_id = -1

session = []
time = 0
last_datetime = datetime.now()

for event in events_list:
    current_datetime = datetime.strptime(event['timestamp'], '%Y-%m-%dT%H:%M:%S')
    time = current_datetime - last_datetime
    last_datetime = current_datetime

    current_session_id = event["session_id"]
    if last_session_id != current_session_id:
        last_session_id = current_session_id
        sessions_list.append([])
        session = sessions_list[-1]
        time = current_datetime - current_datetime

    product = products[event["product_id"]]
    session.append(Event(time, users[event["user_id"]].city, product.price, product.category, event["event_type"], event["offered_discount"]))


In [27]:
with open('records2.json', 'w') as file:
    json.dump([[r.__dict__ for r in records] for records in sessions_list],
              file, indent=2)

print("Zapisano rekordy w pliku json")

Zapisano rekordy w pliku json
